In [1]:
imdb_bert_2k_training_jobs_stats_prefix = "s3://aegovan-data/counterfactuals/imdb/2023042316/"
imdb_bert_2k_pred_date_fmt = "202304232046"
imdb_bert_2k_training_jobs = [
"imdb202304231701-89-00-00-05",
"imdb202304231701-28-00-00-05",
"imdb202304231701-18-00-00-05",
"imdb202304231701-09-00-00-05",
"imdb202304231701-00-00-00-05",
"imdb202304231701-88-00-00-04",
"imdb202304231701-28-00-00-04",
"imdb202304231701-18-00-00-04",
"imdb202304231701-09-00-00-04",
"imdb202304231701-00-00-00-04",
"imdb202304231701-89-00-00-03",
"imdb202304231701-28-00-00-03",
"imdb202304231701-18-00-00-03",
"imdb202304231701-09-00-00-03",
"imdb202304231701-00-00-00-03",
"imdb202304231701-88-00-00-02",
"imdb202304231701-28-00-00-02",
"imdb202304231701-18-00-00-02",
"imdb202304231701-09-00-00-02",
"imdb202304231701-00-00-00-02",
"imdb202304231701-88-00-00-01",
"imdb202304231701-28-00-00-01",
"imdb202304231701-19-00-00-01",
"imdb202304231701-09-00-00-01",
"imdb202304231701-00-00-00-01"
]

In [2]:
imdb_bert_5H_training_jobs_stats_prefix = "s3://aegovan-data/counterfactuals/imdb/202306041250/"

imdb_bert_5H_pred_date_fmt="202306041607"
imdb_bert_5H_training_jobs = [
"imdb202306041410-88-00-00-05",
"imdb202306041410-26-00-00-05",
"imdb202306041410-18-00-00-05",
"imdb202306041410-09-01-00-05",
"imdb202306041410-00-00-00-05",
"imdb202306041410-92-00-00-04",
"imdb202306041410-28-00-00-04",
"imdb202306041410-18-00-00-04",
"imdb202306041410-09-00-00-04",
"imdb202306041410-00-00-00-04",
"imdb202306041410-87-00-00-03",
"imdb202306041410-28-00-00-03",
"imdb202306041410-20-00-00-03",
"imdb202306041410-09-00-00-03",
"imdb202306041410-00-00-00-03",
"imdb202306041410-90-00-00-02",
"imdb202306041410-27-00-00-02",
"imdb202306041410-18-00-00-02",
"imdb202306041410-08-00-00-02",
"imdb202306041410-00-00-00-02",
"imdb202306041410-88-00-00-01",
"imdb202306041410-29-00-00-01",
"imdb202306041410-17-00-00-01",
"imdb202306041410-09-00-00-01",
"imdb202306041410-00-00-00-01"
]



In [3]:
import math

def get_prediction_details(training_jobs,  predicted_date_fmt):
    result = []

    for dataset_type, dataset in {"yelp":"yelp_test.csv.json", "amazon":"amazon_test.csv.json",
                                  "semeval": "semeval_SemEval2017-task4-test.subtask-BD.english.txt.json"}.items():
        for t in training_jobs:
            adv = int(math.ceil(int(t.split("-")[1]) / 10.0)) * 10 
            item = {   "input": f"s3://aegovan-data/counterfactual_{dataset_type}/predictions_{t}/{predicted_date_fmt}/{dataset}",
                       "adv" : adv,
                        "training_job" : t
                    }

            result.append(item)
        
    return result

In [4]:
import sys, os, pandas as pd, boto3
sys.path.append("../src")

import utils

In [5]:
import io

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key

def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)

    s3 = boto3.client('s3')

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read().decode("utf-8")
    
    return io.StringIO(object_content)


In [6]:


import sagemaker, pandas as pd



def load_json_as_df_array(predictions_meta_list):
    result = []
    for meta_data in predictions_meta_list:
        uri = meta_data["input"]
        print(f"Loading {uri}")

        adv = meta_data["adv"]
        df_test = pd.read_json(download_object(uri))
        df_test["uri"] = uri
        df_test["adv"] = adv

        result.append(df_test)
    
    return result





def get_stats_s3(stats_prefix)    :
    s3_stats = []
    for f in filter(lambda x: x.split("/")[-1].startswith("stats_") and "debug" not in x.lower(), 
                    sagemaker.s3.S3Downloader.list(stats_prefix)):

            adv = f.split("/")[-1].split("_")[1]
            item = {
                   "input": f,
                   "adv" : adv
                   }

            s3_stats.append(item)
    return s3_stats



import json
def load_json(predictions_meta_list):
    result = []
    for meta_data in predictions_meta_list:
        uri = meta_data["input"]


        adv = meta_data["adv"]
        item = json.load(download_object(uri))
        item["uri"] = uri
        item["adv"] = adv


        result.append(item)
    
    return result

In [7]:
def get_training_job_details(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_training_job(
    TrainingJobName=job_name
)
    input_data =  response["InputDataConfig"]
    model_details = list(filter(lambda x: x["ChannelName"].upper() == 'PRETRAINED_MODEL', input_data ))[0]
    train_data = list(filter(lambda x: x["ChannelName"].lower() == 'train', input_data ))[0]
    return train_data["DataSource"]["S3DataSource"]["S3Uri"], model_details["DataSource"]["S3DataSource"]["S3Uri"]



In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json 

def _map_train_data_uri_to_stats_uri(train_data_uri):
    
    train_data_parts = train_data_uri.split("/")
    s3_prefix = "/".join( train_data_parts[:-2])
    suffix = "stats_{}_train.json".format( train_data_parts[-2])
    
    s3_stats_uri = "/".join([s3_prefix, suffix])
    return s3_stats_uri
    
def consolidate_predictions_result(df_list, predictions_meta): 
    
    result = []
    for i, (df, meta ) in enumerate(zip(df_list, predictions_meta)):
        print(".")
        training_job = meta["training_job"]
        train_data_uri, model = get_training_job_details(training_job)
        s3_stats_uri =  _map_train_data_uri_to_stats_uri(train_data_uri)
        stats_json = json.load(download_object(s3_stats_uri))
        
        
        accuracy = accuracy_score(df["Sentiment"], df["prediction"])


        prefix = df["uri"].iloc[0].split("/")[-3][-11:].replace("-", "_")
        

        result.append({
            "Train" : "IMDB-C",
            "TrainSize":stats_json["Total"],
            "TrainAffRateP":round(stats_json["AffRateP"],1),
            "TrainAffRateN":round(stats_json["AffRateN"],1),
            "TrainAdvRatePN": round(stats_json["AdvRatePN"],1),
            "TrainPosRate" : round(stats_json["PosRate"],1),
            "TrainUnique":stats_json["Unique"],
            "TrainingJob":training_job,
            "Model" : model.strip("/").split("/")[-1],
            "Test" : df["uri"].iloc[0].split("/")[3],
            "TestSize": "{}K".format(int(round(len(df)/1000, 0))),
            "h_adv_p_n" : df["adv"].iloc[0],
            "Accuracy" : round(accuracy*100, 1),
            "Uri" : df["uri"].iloc[0]


        })
    return pd.DataFrame(result).sort_values(by=["Test", "h_adv_p_n"])

In [9]:

def load_stats_and_results( jobs, pred_datefmt):
    predictions_meta = get_prediction_details(jobs, pred_datefmt)

    predictions_df = load_json_as_df_array(predictions_meta)
    
    df_summary = consolidate_predictions_result(predictions_df, predictions_meta )
    
    return df_summary



In [10]:
# {'Unique': 1999, 'PosRate': 0.5, 'AdvRatePN': 0.89, 'AffRateP': 0.0, 'AffRateN': 0.0, 'Total': 2000}

In [11]:
%%time

predictions_df_bert_2k = load_stats_and_results(
                                            imdb_bert_2k_training_jobs,
                                            imdb_bert_2k_pred_date_fmt
                                            ) 


Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-89-00-00-05/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-28-00-00-05/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-18-00-00-05/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-09-00-00-05/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-00-00-00-05/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-88-00-00-04/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-28-00-00-04/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/predictions_imdb202304231701-18-00-00-04/202304232046/yelp_test.csv.json
Loading s3://aegovan-data/counterfactual_yelp/pr

Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-00-00-00-03/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-88-00-00-02/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-28-00-00-02/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-18-00-00-02/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-09-00-00-02/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/counterfactual_semeval/predictions_imdb202304231701-00-00-00-02/202304232046/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json
Loading s3://aegovan-data/co

mean  \
Model             Test                   TrainSize TrainAdvRatePN          
bert-base-uncased counterfactual_amazon  2000      0.0             88.24   
                                                   0.1             88.92   
                                                   0.2             88.42   
                                                   0.3             88.68   
                                                   0.9             86.90   
                  counterfactual_semeval 2000      0.0             72.98   
                                                   0.1             77.84   
                                                   0.2             70.26   
                                                   0.3             73.66   
                                                   0.9             70.40   
                  counterfactual_yelp    2000      0.0             89.96   
                                                   0.1             90.12   
                                                   0.2             89.90   
                                                   0.3             90.04   
                                                   0.9             86.98   

                                                                        sem  
Model             Test                   TrainSize TrainAdvRatePN            
bert-base-uncased counterfactual_amazon  2000      0.0             0.562672  
                                                   0.1             0.442041  
                                                   0.2             0.336749  
                                                   0.3             0.655286  
                                                   0.9             0.744312  
                  counterfactual_semeval 2000      0.0             3.616960  
                                                   0.1             1.362204  
                                                   0.2             3.114258  
                                                   0.3             3.816753  
                                                   0.9             1.084896  
                  counterfactual_yelp    2000      0.0             0.366879  
                                                   0.1             0.241661  
                                                   0.2             0.291548  
                                                   0.3             0.488467  
                                                   0.9             0.658331

In [13]:
predictions_df_bert_2k.groupby(["Model","Test", "TrainSize", "TrainAdvRatePN"])["Accuracy"].agg(("mean", "sem"))

mean  \
Model             Test                   TrainSize TrainAdvRatePN          
bert-base-uncased counterfactual_amazon  2000      0.0             88.24   
                                                   0.1             88.92   
                                                   0.2             88.42   
                                                   0.3             88.68   
                                                   0.9             86.90   
                  counterfactual_semeval 2000      0.0             72.98   
                                                   0.1             77.84   
                                                   0.2             70.26   
                                                   0.3             73.66   
                                                   0.9             70.40   
                  counterfactual_yelp    2000      0.0             89.96   
                                                   0.1             90.12   
                                                   0.2             89.90   
                                                   0.3             90.04   
                                                   0.9             86.98   

                                                                        sem  
Model             Test                   TrainSize TrainAdvRatePN            
bert-base-uncased counterfactual_amazon  2000      0.0             0.562672  
                                                   0.1             0.442041  
                                                   0.2             0.336749  
                                                   0.3             0.655286  
                                                   0.9             0.744312  
                  counterfactual_semeval 2000      0.0             3.616960  
                                                   0.1             1.362204  
                                                   0.2             3.114258  
                                                   0.3             3.816753  
                                                   0.9             1.084896  
                  counterfactual_yelp    2000      0.0             0.366879  
                                                   0.1             0.241661  
                                                   0.2             0.291548  
                                                   0.3             0.488467  
                                                   0.9             0.658331

In [14]:
import math

def get_display_df(df_scores):

    df_display = df_scores.groupby([ "Train", "Test", "TestSize", "h_adv_p_n"])\
        [["Accuracy"]]\
        .agg(("mean", "std", "max","count" )).reset_index()

    df_display[("Accuracy","stderror")] = df_display[("Accuracy","std")].div(math.sqrt(df_display[("Accuracy","count")].iloc[0]))
    df_display["score"] = df_display["Accuracy"].apply(lambda x: "{:2.1f} ({:3.1f}) ".format( round(x[0],1), round(x[4],1) ), axis=1)
    print(df_display[[ "Train", "Test",  "h_adv_p_n", "score"]].to_latex(index = False))

get_display_df(predictions_df_bert_2k)

\begin{tabular}{llrl}
\toprule
  Train &                    Test & h\_adv\_p\_n &        score \\
        \\
\midrule
 IMDB-C &   counterfactual\_amazon &         0 &  88.2 (0.6)  \\
 IMDB-C &   counterfactual\_amazon &        10 &  88.9 (0.4)  \\
 IMDB-C &   counterfactual\_amazon &        20 &  88.4 (0.3)  \\
 IMDB-C &   counterfactual\_amazon &        30 &  88.7 (0.7)  \\
 IMDB-C &   counterfactual\_amazon &        90 &  86.9 (0.7)  \\
 IMDB-C &  counterfactual\_semeval &         0 &  73.0 (3.6)  \\
 IMDB-C &  counterfactual\_semeval &        10 &  77.8 (1.4)  \\
 IMDB-C &  counterfactual\_semeval &        20 &  70.3 (3.1)  \\
 IMDB-C &  counterfactual\_semeval &        30 &  73.7 (3.8)  \\
 IMDB-C &  counterfactual\_semeval &        90 &  70.4 (1.1)  \\
 IMDB-C &     counterfactual\_yelp &         0 &  90.0 (0.4)  \\
 IMDB-C &     counterfactual\_yelp &        10 &  90.1 (0.2)  \\
 IMDB-C &     counterfactual\_yelp &        20 &  89.9 (0.3)  \\
 IMDB-C &     counterfactual\_yelp & 